# Выбор локации для скважины

**Описание задачи:**

Нефтедобывающей компании нужно решить, где бурить новую скважину.

Для анализа предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерено качество нефти и объём её запасов. 

Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Возможная прибыль и риски должны быть проанализированы техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Условия задачи:**

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые);
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки;
- Бюджет на разработку скважин в регионе — 10 млрд рублей;
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей;
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Обучение-и-проверка-моделей" data-toc-modified-id="Обучение-и-проверка-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка моделей</a></span><ul class="toc-item"><li><span><a href="#Регион-1" data-toc-modified-id="Регион-1-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Регион-1</a></span></li><li><span><a href="#Регион-2" data-toc-modified-id="Регион-2-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Регион-2</a></span></li><li><span><a href="#Регион-3" data-toc-modified-id="Регион-3-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Регион-3</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Расчёт-прибыли" data-toc-modified-id="Расчёт-прибыли-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Расчёт прибыли</a></span></li><li><span><a href="#Расчёт-рисков" data-toc-modified-id="Расчёт-рисков-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Расчёт рисков</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Загрузка и подготовка данных

### Загрузка данных

Импортируем необходимые для работы библиотеки:

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

Загрузим данные:

In [101]:
geodata_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
geodata_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
geodata_3 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

### Изучение данных

Выведем на экран первые 5 строк каждого из датасетов, а также общую информацию о данных:

In [102]:
display(geodata_1.head())
geodata_1.info()
display(geodata_2.head())
geodata_2.info()
display(geodata_3.head())
geodata_3.info()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Проверим, есть ли в датасетах грубые дубликаты:

In [103]:
print(geodata_1.duplicated().sum())
print(geodata_2.duplicated().sum())
print(geodata_3.duplicated().sum())

0
0
0


###### Вывод

Рассмотрены три датасета, содержащих данные геологоразведки трёх регионов.

В каждом датасете по 5 столбцов со следующими типами данных: *float64, object*.

Каждый объект в наборе данных — это характеристики скважин. Известно:

- ***id*** — уникальный идентификатор скважины;
- ***f0, f1, f2*** — признаки точек неизвестного назначения, но по условиям задачи считающиеся значимыми;
- ***product*** — объём запасов в скважине, тыс. баррелей.

Столбцы ***id*** вряд ли как-то связаны с объемом запасов нефти в скважинах, поэтому при дальнейшей обработке исключим данные столбцы из выборок.

Пропуски в датасетах отсутствуют.

Грубые дубликаты в датасетах отсутствуют.

### Подготовка данных

Удалим из датасетов столбцы с уникальным идентификатором скважин:

In [104]:
geodata_1 = geodata_1.drop('id', axis = 1)
geodata_2 = geodata_2.drop('id', axis = 1)
geodata_3 = geodata_3.drop('id', axis = 1)

Выведем на экран первые строчки обновленных датасетов:

In [105]:
display(geodata_1.head(1))
display(geodata_2.head(1))
display(geodata_3.head(1))

,f0,f1,f2,product
0,0.705745,-0.497823,1.22117,105.280062


,f0,f1,f2,product
0,-15.001348,-8.276,-0.005876,3.179103


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673


Столбцы ***id*** успешно удалены.

В датасетах ***geodata_0***,***geodata_1***,***geodata_2***, столбец ***product*** содержит информацию об объеме запасов нефти в скважинах. Выделим данный столбец в целевой признак: задачей обученной модели будет предсказание объёмов запаса нефти в скважинах. 

Выделим признаки:

In [106]:
features_geo1 = geodata_1.drop(['product'], axis = 1)
features_geo2 = geodata_2.drop(['product'], axis = 1)
features_geo3 = geodata_3.drop(['product'], axis = 1)

target_geo1 = geodata_1['product']
target_geo2 = geodata_2['product']
target_geo3 = geodata_3['product']

Для контроля переобучения и правильной оценки готовой модели разделим данные каждого региона на обучающие и валидационные выборки в следующем соотношении:

 - Обучающая выборка: 75%;
 - Валидационная выборка: 25%.

Напишем функцию для разделения признаков на обучающие и валидационные:

In [107]:
def split_data(features, target):
    
    features_train,features_valid,target_train,target_valid = train_test_split(
        features,
        target,
        test_size = 0.25,
        random_state = 12345)
    
    return features_train,features_valid,target_train,target_valid

Разделим признаки на обучающие и валидационные

In [108]:
features_train_geo1, features_valid_geo1, target_train_geo1, target_valid_geo1 = split_data(
    features_geo1,
    target_geo1
)

features_train_geo2, features_valid_geo2, target_train_geo2, target_valid_geo2 = split_data(
    features_geo2,
    target_geo2
)

features_train_geo3, features_valid_geo3, target_train_geo3, target_valid_geo3 = split_data(
    features_geo3,
    target_geo3
)

Проверим правильность разделения признаков на примере данных первого региона:

In [109]:
print('Размер обучающей таблицы признаков:',features_train_geo1.shape)
print('Размер обучающей таблицы с целевым признаком:',target_train_geo1.shape)
print('Размер валидационной таблицы признаков:',features_valid_geo1.shape)
print('Размер валидационной таблицы с целевым признаком:',target_valid_geo1.shape)

Размер обучающей таблицы признаков: (75000, 3)
Размер обучающей таблицы с целевым признаком: (75000,)
Размер валидационной таблицы признаков: (25000, 3)
Размер валидационной таблицы с целевым признаком: (25000,)


Алгоритм, с которым мы будем работать, ***Линейная регрессия***, относится к линейным алгоритмам. При разном масштабе признаков линейные алгоритмы обучения могут сильнее выделять те или иные признаки. Для того, чтобы алгоритм воспринимал значимость каждого из признаков одинаково, необходимо их стандартизировать. Проведем стандартизацию:

Напишем функцию для стандартизации признаков:

In [110]:
scaler = StandardScaler()

def standardize(features_train, features_valid):
    
    scaler.fit(features_train)
    features_train = pd.DataFrame(
        scaler.transform(features_train),
        columns = features_geo1.columns
    )
    
    features_valid = pd.DataFrame(
        scaler.transform(features_valid),
        columns = features_geo1.columns
    )
    
    return features_train, features_valid

Стандартизируем признаки каждого из регионов:

In [111]:
features_train_geo1,features_valid_geo1 = standardize(
    features_train_geo1,
    features_valid_geo1
)

features_train_geo2,features_valid_geo2 = standardize(
    features_train_geo2,
    features_valid_geo2
)

features_train_geo3,features_valid_geo3 = standardize(
    features_train_geo3,
    features_valid_geo3
)

In [112]:
display(features_train_geo1.describe())
display(features_train_geo2.describe())
display(features_train_geo3.describe())

,f0,f1,f2
count,7.500000e+04,7.500000e+04,7.500000e+04
mean,1.704118e-16,-3.349025e-17,6.252776e-18
std,1.000007e+00,1.000007e+00,1.000007e+00
min,-2.186288e+00,-2.178269e+00,-3.890945e+00
25%,-6.575544e-01,-8.925344e-01,-6.799576e-01
50%,1.878842e-03,-7.407961e-04,4.288679e-03
75%,6.571176e-01,8.928514e-01,6.829752e-01
max,2.139081e+00,2.169192e+00,4.153623e+00


,f0,f1,f2
count,7.500000e+04,7.500000e+04,7.500000e+04
mean,5.684342e-19,-1.028392e-16,-2.244131e-17
std,1.000007e+00,1.000007e+00,1.000007e+00
min,-3.652789e+00,-4.206546e+00,-1.476535e+00
25%,-8.290577e-01,-6.766541e-01,-8.784844e-01
50%,1.734177e-03,-1.083306e-03,-2.838957e-01
75%,8.349682e-01,6.768381e-01,8.834721e-01
max,3.154487e+00,4.588691e+00,1.482457e+00


,f0,f1,f2
count,7.500000e+04,7.500000e+04,7.500000e+04
mean,-2.197946e-17,6.631732e-18,-8.517039e-17
std,1.000007e+00,1.000007e+00,1.000007e+00
min,-5.060306e+00,-4.088636e+00,-4.167323e+00
25%,-6.716773e-01,-6.783003e-01,-6.785573e-01
50%,3.845213e-03,-5.219873e-03,-5.092214e-03
75%,6.656278e-01,6.749474e-01,6.802621e-01
max,4.176529e+00,4.533914e+00,4.101780e+00


Признаки успешно стандартизированы: средние значения признаков приближены к нулю, стандартные отклонения к единице.

###### Вывод

Из датасетов удалены столбцы ***id*** в связи с их незначимостью в качестве признаков.

Данные каждого региона разделены на обучающие и валидационные выбоки. Размеры выборок соответствуют соотношению:
 - Обучающая выборка: 75%;
 - Валидационная выборка: 25%.

С целью одинакового восприятия алгоритмом значимости каждого из признаков, проведена их стандартизация для каждого из регионов.


## Обучение и проверка моделей

Напишем функцию для обучения моделей и сохранения предсказаний и ответов по региону в датафреймы:

In [113]:
def get_predictions(
    features_train,
    features_valid,
    target_train,
    target_valid
):
        
    reg = LinearRegression()
        
    reg.fit(features_train, target_train)
        
    predictions_train = reg.predict(features_train)
    predictions_valid = reg.predict(features_valid)
    
    stats = pd.DataFrame(
        {'target':target_valid.reset_index(drop=True),
         'predictions':predictions_valid}
    )
    
    return stats

### Регион-1

Обучим модель на данных первого региона и сохраним целевые и предсказанные данные в переменной ***stat_geo_1***:

In [114]:
stat_geo1 = get_predictions(
    features_train_geo1,
    features_valid_geo1,
    target_train_geo1,
    target_valid_geo1
)

display(stat_geo1.head())

print('R2:',r2_score(target_valid_geo1,stat_geo1['predictions']).round(2))
print('RMSE:',(mean_squared_error(target_valid_geo1,stat_geo1['predictions']) ** 0.5).round(2))
print('Средний запас предсказанного сырья:',stat_geo1['predictions'].mean().round(2))

,target,predictions
0,10.038645,95.894952
1,114.551489,77.572583
2,132.603635,77.892640
3,169.072125,90.175134
4,122.325180,70.510088


R2: 0.28
RMSE: 37.58
Средний запас предсказанного сырья: 92.59


### Регион-2

Обучим модель на данных второго региона и сохраним целевые и предсказанные данные в переменной ***stat_geo_2***:

In [115]:
stat_geo2 = get_predictions(
    features_train_geo2,
    features_valid_geo2,
    target_train_geo2,
    target_valid_geo2
)

display(stat_geo2.head())

print('R2:',r2_score(target_valid_geo2,stat_geo2['predictions']).round(2))
print('RMSE:',(mean_squared_error(target_valid_geo2,stat_geo2['predictions']) ** 0.5).round(2))
print('Средний запас предсказанного сырья:',stat_geo2['predictions'].mean().round(2))

,target,predictions
0,80.859783,82.663314
1,53.906522,54.431786
2,30.132364,29.748760
3,53.906522,53.552133
4,0.000000,1.243856


R2: 1.0
RMSE: 0.89
Средний запас предсказанного сырья: 68.73


### Регион-3

Обучим модель на данных третьего региона и сохраним целевые и предсказанные данные в переменной ***stat_geo_3***:

In [116]:
stat_geo3 = get_predictions(
    features_train_geo3,
    features_valid_geo3,
    target_train_geo3,
    target_valid_geo3
)

display(stat_geo3.head())

print('R2:',r2_score(target_valid_geo3,stat_geo3['predictions']).round(2))
print('RMSE:',(mean_squared_error(target_valid_geo3,stat_geo3['predictions']) ** 0.5).round(2))
print('Средний запас предсказанного сырья:',stat_geo3['predictions'].mean().round(2))

,target,predictions
0,61.212375,93.599633
1,41.850118,75.105159
2,57.776581,90.066809
3,100.053761,105.162375
4,109.897122,115.303310


R2: 0.21
RMSE: 40.03
Средний запас предсказанного сырья: 94.97


###### Вывод

На этапе обучения и проверки моделей для каждого региона была обучена модель с помощью алгоритма ***Линейной регрессии***.

***Метрики качества*** обученных моделей следующие:

***Регион 1:***

- ***R2***: 0.28
- ***RMSE***: 37.58

***Регион 2:***

- ***R2***: 1.00
- ***RMSE***: 0.89

***Регион 3:***

- ***R2***: 0.21
- ***RMSE***: 40.03

Из трех моделей наиболее качественной и точной является модель для региона 2. Коэффициент детерминации, ***R2***, данной модели равен ***1.00***, что говорит о практически функциональной зависимости между признаками. Значение ***RMSE*** данной модели говорит о том, что, в среднем, отклонение между наблюдаемыми и истинными значениями равно ***0.89 тыс. баррелей***.

Модели для регионов 1 и 2 гораздо ниже по качеству и точности. Коэффициенты детерминации,***R2***, равные ***0.28*** и ***0.21*** соответсвенно, говорят о том, что модели ненамного лучше констатных, предсказывающих средние значения. Значения ***RMSE*** моделей говорят о том, что, в среднем, отклонения между наблюдаемыми и истинными значениями равны ***37.58 и 40.03 тыс. баррелей*** соответственно.

Несмотря на низкое качество моделей для регионов 1 и 2, мы продолжим работу с результатами их предсказаний, так как данные модели нужны только для того, чтобы выбрать самые перспективные скважины. То есть, несмотря на то, что модели могут сильно ошибаться в объеме запасов в каждой скважине, они будут использованы для выбора ТОП-скважин по объему запасов.

***Средние запасы предсказанного сырья*** в скважине по регионам следующие:

 - ***Регион 1:*** ***92.59*** тыс. баррелей;
 - ***Регион 2:*** ***68.73*** тыс. баррелей;
 - ***Регион 3:*** ***94.97*** тыс. баррелей.
 
Наибольший средний запас предсказанного сырья в скважине наблюдается в регионе 1, и это незначительно превышает  показатель по региону 2.
Наименьший средний запас предсказанного сырья в скважине наблюдается в регионе 2.

## Подготовка к расчёту прибыли

Бюджет на разработку скважин в регионе — ***10 млрд. рублей***. Планируется разработать ***200 скважин***. Рассчитаем бюджет на разработку 1 скважины в тыс. рублей:

In [117]:
SUM_BUDGET = 10

In [118]:
WELL_BUDGET = SUM_BUDGET / 200
print('Бюджет на разработку одной скважины: {:.3f} млрд. рублей.'.format(WELL_BUDGET))

Бюджет на разработку одной скважины: 0.050 млрд. рублей.


Цена ***1000 баррелей*** нефти составляет ***450 тыс. рублей***. Рассчитаем объем добываемого продукта для безубыточной разработки скважины:

In [119]:
BARREL_1K_PRICE = 450
BREAK_EVEN = 50000 / BARREL_1K_PRICE
print('Объем продукта для безубыточной разработки скважины: {:.2f} тыс. баррелей.'.format(BREAK_EVEN))

Объем продукта для безубыточной разработки скважины: 111.11 тыс. баррелей.


###### Вывод

Объем продукта для безубыточной разработки скважины = ***111,11 тыс. баррелей***.

Ни один из средних запасов предсказанного сырья в скважинах, оцененных по каждому региону, не достаточен для безубыточной разработки.

Таким образом, далее, при расчете потенциальной прибыли, мы будем использовать ТОПы скважин по предсказанным объемам запасов в каждом из регионов.

## Расчёт прибыли и рисков 

### Расчёт прибыли

Напишем функцию для расчета среднего запаса сырья, средней прибыли и суммарной прибыли от добычи нефти из ТОП-200 скважин по объемам запасов сырья в регионах:

In [120]:
def top200(stat_geo):
    predictions_sorted = stat_geo.sort_values(by = 'predictions', ascending=False)
    selected = predictions_sorted['target'][:200]
    mean_volume = selected.mean()
    mean_revenue = selected.mean() * BARREL_1K_PRICE / 1000000 - WELL_BUDGET
    revenue = selected.sum() * BARREL_1K_PRICE / 1000000 - SUM_BUDGET
    
    return mean_volume, mean_revenue, revenue

Рассчитаем прибыль c ТОП-200 скважин по запасам сырья в каждом регионе:

In [121]:
top_mean_volume_reg_1 = top200(stat_geo1)[0]
top_mean_revenue_reg_1 = top200(stat_geo1)[1]
revenue_reg_1 = top200(stat_geo1)[2]

top_mean_volume_reg_2 = top200(stat_geo2)[0]
top_mean_revenue_reg_2 = top200(stat_geo2)[1]
revenue_reg_2 = top200(stat_geo2)[2]

top_mean_volume_reg_3 = top200(stat_geo3)[0]
top_mean_revenue_reg_3 = top200(stat_geo3)[1]
revenue_reg_3 = top200(stat_geo3)[2]

for region, volume, mean_revenue, revenue in [('Регион-1',top_mean_volume_reg_1,top_mean_revenue_reg_1,revenue_reg_1),
                                              ('Регион-2',top_mean_volume_reg_2,top_mean_revenue_reg_2,revenue_reg_2),
                                              ('Регион-3',top_mean_volume_reg_3,top_mean_revenue_reg_3,revenue_reg_3)
                                             ]:
    print('{}:'.format(region))
    print('Средний объем запасов в скважине для ТОП-200: {:.2f}, тыс. баррелей'.format(volume))
    print('Средняя прибыль от добычи нефти из скважины для ТОП-200: {:.3f}, млрд. рублей'.format(mean_revenue))
    print('Прибыль от добычи нефти из ТОП-200 скважин: {:.2f}, млрд. рублей'.format(revenue))
    print()

Регион-1:
Средний объем запасов в скважине для ТОП-200: 148.01, тыс. баррелей
Средняя прибыль от добычи нефти из скважины для ТОП-200: 0.017, млрд. рублей
Прибыль от добычи нефти из ТОП-200 скважин: 3.32, млрд. рублей

Регион-2:
Средний объем запасов в скважине для ТОП-200: 137.95, тыс. баррелей
Средняя прибыль от добычи нефти из скважины для ТОП-200: 0.012, млрд. рублей
Прибыль от добычи нефти из ТОП-200 скважин: 2.42, млрд. рублей

Регион-3:
Средний объем запасов в скважине для ТОП-200: 141.23, тыс. баррелей
Средняя прибыль от добычи нефти из скважины для ТОП-200: 0.014, млрд. рублей
Прибыль от добычи нефти из ТОП-200 скважин: 2.71, млрд. рублей



###### Вывод

Из произведенных расчетов видно, что:

- ТОП-200 скважин каждой из валидационных выборок обладает достаточным средним запасом сырья в скважине для прибыльной добычи нефти;


- ***Наибольшую*** прибыль приносит добыча нефти из ТОП-200 скважин по запасам сырья ***Региона-1***, где средний объем сырья в скважине = ***148.01 тыс баррелей***, средняя прибыль с одной скважины = ***0.017 млрд. рублей***, суммарная прибыль с ТОП-200 скважин = ***3.32 млрд. рублей***;


- ***Наименьшую*** прибыль приносит добыча нефти из ТОП-200 скважин по запасам сырья ***Региона-2***, где средний объем сырья в скважине = ***137.95 тыс баррелей***, средняя прибыль с одной скважины = ***0.012 млрд. рублей***, суммарная прибыль с ТОП-200 скважин = ***2.42 млрд. рублей***.

### Расчёт рисков

Ранее мы рассчитали прибыль для ТОП-200 скважин из валидационных выборок по кадому региону, т.е. для 200 лучших скважин по объему сырья из 25000 скважин в каждом регионе.

По условиям задачи при разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

Так как выбор 500 точек для исследования в каждом регионе никак не обоснован, будем считать этот выбор случайным. В таком случае, нам необходимо убедиться, что при любом случайном выборе 500 точек в каждом регионе, ТОП-200 скважин по-прежнему будет прибыльным.

По условиям задачи вероятность убытков от разработки месторождения не должна превысить 2,5%. Также, нам необходимо определить 95% доверительный интервал. 

Для удовлетворения всех вышеописанных условий для нахождения распределения прибыли применим технику ***Bootstrap*** с 1000 выборок.

Напишем функцию для нахождения распределения среднего запаса сырья скважины, средней прибыли со скважины и суммарной прибыли с ТОП-200 скважин из случайных 500 с помощью ***Bootstrap***, а также для определения вероятности убытков:

In [122]:
def identify_risks(stat_geo):
    
    state = np.random.RandomState(12345)
    
    volume_per_well = []
    revenue_per_well = []
    revenue = []
    
    for i in range(1000):
        
        sample = stat_geo.sample(n=500, replace=True, random_state=state)
        volume_per_well.append(top200(sample)[0])
        revenue_per_well.append(top200(sample)[1])
        revenue.append(top200(sample)[2])
    
    volume_per_well = pd.Series(volume_per_well)
    revenue_per_well = pd.Series(revenue_per_well)
    revenue = pd.Series(revenue)
    
    lower_volume_per_well = volume_per_well.quantile(0.025)
    lower_revenue_per_well = revenue_per_well.quantile(0.025)
    lower_revenue = revenue.quantile(0.025)

    upper_volume_per_well = volume_per_well.quantile(0.975)
    upper_revenue_per_well = revenue_per_well.quantile(0.975)
    upper_revenue = revenue.quantile(0.975)
    
    mean_volume_per_well = volume_per_well.mean()
    mean_revenue_per_well = revenue_per_well.mean()
    mean_revenue = revenue.mean()
    
    loss_probability = (lambda x: x[x < 0].count() / len(x))(revenue) * 100

    results = pd.DataFrame(
        {
            'Показатель': ['Средний объем запасов в скважине, тыс. баррелей',
                           'Средняя прибыль от скважины, млрд. рублей',
                           'Суммарная прибыль, млрд. рублей'],
            '2,5% квантиль': [lower_volume_per_well,
                              lower_revenue_per_well,
                              lower_revenue],
            'Среднее': [mean_volume_per_well,
                        mean_revenue_per_well,
                        mean_revenue],
            '97,5% квантиль': [upper_volume_per_well,
                             upper_revenue_per_well,
                             upper_revenue] 
        }
    ).set_index('Показатель')
    
    display(results)
    print('Вероятность убытков:',loss_probability, '%')
    print()

Рассчитаем доверительные интервалы для среднего запаса сырья скважины, средней прибыли со скважины и суммарной прибыли с ТОП-200 скважин, применив написанную функцию, а также определим вероятность убытков для каждого региона:

In [123]:
for region, stat_geo in [('Регион-1',stat_geo1),
                         ('Регион-2',stat_geo2),
                         ('Регион-3',stat_geo3)
                        ]:
    print (region)
    identify_risks(stat_geo)

Регион-1


,"2,5% квантиль",Среднее,"97,5% квантиль"
Показатель,,,
"Средний объем запасов в скважине, тыс. баррелей",109.875383,115.512944,121.219633
"Средняя прибыль от скважины, млрд. рублей",-0.000556,0.001981,0.004549
"Суммарная прибыль, млрд. рублей",-0.111216,0.396165,0.909767


Вероятность убытков: 6.9 %

Регион-2


,"2,5% квантиль",Среднее,"97,5% квантиль"
Показатель,,,
"Средний объем запасов в скважине, тыс. баррелей",111.486895,116.178279,120.580994
"Средняя прибыль от скважины, млрд. рублей",0.000169,0.002280,0.004261
"Суммарная прибыль, млрд. рублей",0.033821,0.456045,0.852289


Вероятность убытков: 1.5 %

Регион-3


,"2,5% квантиль",Среднее,"97,5% квантиль"
Показатель,,,
"Средний объем запасов в скважине, тыс. баррелей",109.296107,115.604487,121.670662
"Средняя прибыль от скважины, млрд. рублей",-0.000817,0.002022,0.004752
"Суммарная прибыль, млрд. рублей",-0.163350,0.404404,0.950360


Вероятность убытков: 7.6 %



###### Вывод

С ***вероятностью 95%*** показатели ТОП-200 скважин из случайных 500 в регионах ожидаются следующие:

**Регион-1**

- Средний объем запасов в скважине: в среднем ***115.51*** (от 109.88 до 121.22) тыс. баррелей;
- Средняя прибыль от скважины: в среднем ***1.98*** (от -0.556 до 4,55) млн. рублей;
- Суммарная прибыль: в среднем ***0,40*** (от -0.11 до 0.91) млрд. рублей.

С ***вероятностью 6.9%*** суммарная прибыль от разработки региона может быть отрицательной, что ***не соответствует*** условию задачи на согласование разработки региона. 

**Регион-2**

- Средний объем запасов в скважине: в среднем ***116.18*** (от 111.49 до 120.58) тыс. баррелей;
- Средняя прибыль от скважины: в среднем ***2.28*** (от 0.17 до 4.26) млн. рублей;
- Суммарная прибыль: в среднем ***0,46*** (от 0.03 до 0.85) млрд. рублей.

С ***вероятностью 1,5%*** суммарная прибыль от разработки региона может быть отрицательной, что ***соответствует*** условию задачи на согласование разработки региона. 

**Регион-3**

- Средний объем запасов в скважине: в среднем ***115.60*** (от 109.30 до 121.67) тыс. баррелей;
- Средняя прибыль от скважины: в среднем ***2.02*** (от -0.82 до 4.75) млн. рублей;
- Суммарная прибыль: в среднем ***0,40*** (от -0.16 до 0.95) млрд. рублей.

С ***вероятностью выше 7,6%*** суммарная прибыль от разработки региона может быть отрицательной, что ***не соответствует*** условию задачи на согласование разработки региона. 

## Общий вывод

Подходящим для разработки считается регион, который после разработки в течение срока эксплуатации окупит ***10 млрд. рублей***, заложенных в бюджет, а также ***вероятность получения убытка*** в котором составит ***не более 2,5%***.

Исходя из полученных результатов, регионом, полностью соответствующим условиям задачи на разработку является ***Регион-2***.

***Вероятность получения убытков*** от разработки региона ***не более 1,5%***: ***минимальная прибыль*** в 95% доверительном интервале = ***0.03 млрд. рублей*** (против -0.11 и -0.16 млрд. в регионах 1 и 3 соответственно).

***Средняя прибыль*** в 95% доверительном интервале = ***0.46 млрд. рублей*** (против 0.4 млрд. в регионах 1 и 3), что ***позволяет получить в среднем 460 млн. рублей*** прибыли.